In [ ]:
import os
import sys

# 1. 创建源码目录
os.makedirs("src", exist_ok=True)
sys.path.append("/kaggle/working/src")

# 2. 自动寻找数据根目录
DATA_ROOT = ""
# 遍历 input 目录寻找特征标志文件
for root, dirs, files in os.walk("/kaggle/input"):
    if "train_map_300.txt" in files:
        DATA_ROOT = root
        break

if DATA_ROOT:
    print(f"✅ 成功定位数据根目录: {DATA_ROOT}")
    print("   包含文件示例:", os.listdir(DATA_ROOT)[:5])
    
    # 检查关键的归一化文件是否存在
    if "global_mean_300_double_vel.npy" in os.listdir(DATA_ROOT):
        print("✅ 检测到双重相对坐标统计量 (Double Relative Stats)")
    else:
        print("⚠️ 警告：未找到 global_mean_300_double_vel.npy，请检查数据集！")
else:
    print("❌ 错误: 未找到数据集，请检查是否 Add Input")

In [ ]:
%%writefile src/config.py
import os

# ================= 路径配置 =================
RESULT_DIR = "/kaggle/working/results"

# ================= 数据参数 =================
# 原始 MP 特征是 134，经过双重相对+速度处理后变成 268
INPUT_SIZE = 268     
SEQ_LEN = 64         
NUM_CLASSES = 300    

# ================= 训练参数 =================
BATCH_SIZE = 64      # P100 显存足够
EPOCHS = 80
LEARNING_RATE = 1e-3
DEVICE = "cuda"

In [ ]:
%%writefile src/model.py
import torch
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        # MLP Attention: Linear -> Tanh -> Linear
        self.attn = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, lstm_output):
        # lstm_output: [Batch, Seq, Hidden*2]
        scores = self.attn(lstm_output)
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * lstm_output, dim=1)
        return context

class BiLSTMAttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=2, dropout=0.5):
        super(BiLSTMAttentionModel, self).__init__()
        
        # 1. BiLSTM
        # ❗ 修复1: LSTM dropout 强制设为 0.2，防止层间信息丢失过多导致梯度断裂
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.2 
        )
        
        # 2. LayerNorm
        self.ln = nn.LayerNorm(hidden_size * 2)
        
        # 3. Attention
        self.attention = Attention(hidden_size)

        # 4. Classifier Head
        # ❗ 修复2: Classifier dropout 设为 0.3，防止欠拟合
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3), 
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        out = self.ln(out)
        context = self.attention(out)
        output = self.classifier(context)
        return output

In [ ]:
%%writefile src/dataset.py
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import random

# ==========================================
# 核心特征工程：双重相对坐标 + 速度
# ==========================================
def feature_engineering(data):
    """
    Input: (T, 134) Absolute Coordinates
    Output: (T, 268) Double Relative + Velocity
    """
    T = data.shape[0]
    
    pose = data[:, 0:50].reshape(T, 25, 2)
    lh   = data[:, 50:92].reshape(T, 21, 2)
    rh   = data[:, 92:134].reshape(T, 21, 2)

    nose = pose[:, 0:1, :]      
    l_wrist = lh[:, 0:1, :]     
    r_wrist = rh[:, 0:1, :]     

    pose_rel = pose - nose
    lh_rel = lh - l_wrist
    rh_rel = rh - r_wrist

    pose_d = np.diff(pose_rel, axis=0, prepend=pose_rel[:1])
    lh_d   = np.diff(lh_rel,   axis=0, prepend=lh_rel[:1])
    rh_d   = np.diff(rh_rel,   axis=0, prepend=rh_rel[:1])

    final_feat = np.concatenate([
        pose_rel.reshape(T, -1),
        lh_rel.reshape(T, -1),
        rh_rel.reshape(T, -1),
        pose_d.reshape(T, -1),
        lh_d.reshape(T, -1),
        rh_d.reshape(T, -1)
    ], axis=1)

    return final_feat

class WLASLDataset(Dataset):
    def __init__(self, map_file, data_root, mode='train', seq_len=64):
        self.mode = mode
        self.data_root = data_root
        self.seq_len = seq_len
        
        with open(map_file, 'r') as f:
            self.lines = f.readlines()
            
        mean_path = os.path.join(data_root, "global_mean_300_double_vel.npy")
        std_path = os.path.join(data_root, "global_std_300_double_vel.npy")
        
        # 路径容错
        if not os.path.exists(mean_path):
             for root, dirs, files in os.walk(data_root):
                if "global_mean_300_double_vel.npy" in files:
                    mean_path = os.path.join(root, "global_mean_300_double_vel.npy")
                    std_path = os.path.join(root, "global_std_300_double_vel.npy")
                    break

        if os.path.exists(mean_path):
            self.mean = np.load(mean_path).astype(np.float32)
            self.std = np.load(std_path).astype(np.float32)
        else:
            self.mean = None
            
    def __len__(self):
        return len(self.lines)
    
    def __getitem__(self, idx):
        # ❗ 修复4: 防止无限递归死循环，尝试最多 10 次
        max_retries = 10
        for _ in range(max_retries):
            try:
                return self._load_item(idx)
            except Exception:
                # 如果读取失败，尝试下一个索引
                idx = (idx + 1) % len(self)
        
        # 如果连续 10 个都坏了，返回全 0 Tensor (Fallback)
        return torch.zeros((self.seq_len, 268), dtype=torch.float32), torch.tensor(0, dtype=torch.long)

    def _load_item(self, idx):
        line = self.lines[idx].strip()
        original_path, label = line.split(',')
        filename = os.path.basename(original_path)
        
        npy_path = os.path.join(self.data_root, "processed_features_300", filename)
        if not os.path.exists(npy_path):
            found = False
            for root, dirs, files in os.walk(self.data_root):
                if filename in files:
                    npy_path = os.path.join(root, filename)
                    found = True
                    break
            if not found:
                raise FileNotFoundError(f"File not found: {filename}")

        # 1. 加载
        raw_data = np.load(npy_path).astype(np.float32)
        
        # 2. 特征工程 (134 -> 268)
        data = feature_engineering(raw_data)
        
        # 3. 归一化
        if self.mean is not None:
            data = (data - self.mean) / self.std
            
        # ❗ 修复3: 噪声必须在归一化之后加，且强度设为 0.02
        if self.mode == 'train':
            noise = np.random.normal(0, 0.02, data.shape).astype(np.float32)
            data += noise
        
        # 4. 长度统一
        curr_len = data.shape[0]
        feature_dim = data.shape[1]
        if curr_len > self.seq_len:
            indices = np.linspace(0, curr_len-1, self.seq_len, dtype=int)
            data = data[indices]
        elif curr_len < self.seq_len:
            pad = np.zeros((self.seq_len - curr_len, feature_dim), dtype=np.float32)
            data = np.concatenate([data, pad], axis=0)

        return torch.from_numpy(data), torch.tensor(int(label), dtype=torch.long)

In [ ]:
%%writefile src/train.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import random

from dataset import WLASLDataset
from model import BiLSTMAttentionModel
import config

torch.backends.cudnn.benchmark = True

# Worker 初始化
def worker_init_fn(worker_id):
    seed = torch.initial_seed() % 2**32
    np.random.seed(seed)
    random.seed(seed)

def plot_results(history, save_dir):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history['val_acc'], label='Val Acc')
    plt.plot(history['train_acc'], label='Train Acc')
    plt.title('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(save_dir, "training_curves.png"))
    plt.close()

def evaluate_topk(model, dataloader, device, k=5):
    model.eval()
    correct_1 = 0
    correct_k = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            out = model(x)
            
            _, pred_1 = torch.max(out, 1)
            correct_1 += (pred_1 == y).sum().item()
            
            _, pred_k = out.topk(k, 1, True, True)
            correct_k += pred_k.eq(y.view(-1, 1).expand_as(pred_k)).sum().item()
            
            total += y.size(0)
            all_preds.extend(pred_1.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
            
    return correct_1/total, correct_k/total, all_labels, all_preds

def main(data_root):
    os.makedirs(config.RESULT_DIR, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🔥 Device: {device}")
    
    print(f"📦 Loading Data from: {data_root}")
    train_ds = WLASLDataset(os.path.join(data_root, "train_map_300.txt"), data_root, 'train', config.SEQ_LEN)
    val_ds = WLASLDataset(os.path.join(data_root, "val_map_300.txt"), data_root, 'val', config.SEQ_LEN)
    test_ds = WLASLDataset(os.path.join(data_root, "test_map_300.txt"), data_root, 'test', config.SEQ_LEN)
    
    # ❗ 修复5: 所有 DataLoader 都加上 worker_init_fn
    loader_kwargs = {
        "batch_size": config.BATCH_SIZE,
        "num_workers": 4,  # Kaggle P100 上 4 比较稳，如果卡死改成 2
        "pin_memory": True,
        "persistent_workers": True,
        "worker_init_fn": worker_init_fn
    }
    
    train_loader = DataLoader(train_ds, shuffle=True, **loader_kwargs)
    val_loader = DataLoader(val_ds, shuffle=False, **loader_kwargs)
    test_loader = DataLoader(test_ds, shuffle=False, **loader_kwargs)
    
    print(f"✅ Data Loaded: Train={len(train_ds)}, Val={len(val_ds)}, Test={len(test_ds)}")
    
    model = BiLSTMAttentionModel(
        config.INPUT_SIZE, 
        hidden_size=512, 
        num_classes=config.NUM_CLASSES,
        dropout=0.5 # 这个参数在 model.py 里虽然传进去了，但 model.py 内部已经修正为 0.2 和 0.3
    ).to(device)
    
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)
    
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}
    save_path = os.path.join(config.RESULT_DIR, "best_model_300.pth")
    
    print("🚀 Starting Training...")
    for epoch in range(config.EPOCHS):
        model.train()
        train_loss, correct, total = 0, 0, 0
        
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.EPOCHS}", leave=False):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, pred = torch.max(out, 1)
            total += y.size(0)
            correct += (pred == y).sum().item()
            
        train_acc = correct / total
        avg_loss = train_loss / len(train_loader)
        
        val_acc, val_acc5, _, _ = evaluate_topk(model, val_loader, device, k=5)
        
        history['train_loss'].append(avg_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        scheduler.step(val_acc)
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            # print(f"   ⭐ New Best Val Acc: {val_acc:.4f}")
            
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f} | Train={train_acc:.2%} | Val={val_acc:.2%} (Top5={val_acc5:.2%})")

    # Final Test
    print("\n🧪 Final Evaluation on Test Set...")
    if os.path.exists(save_path):
        # ❗ 修复6: 加载模型时加上 map_location，防止设备不一致报错
        model.load_state_dict(torch.load(save_path, map_location=device))
    
    acc_1, acc_5, y_true, y_pred = evaluate_topk(model, test_loader, device, k=5)
    print(f"\n🏆 Final Test Results:")
    print(f"   Top-1 Accuracy: {acc_1:.2%}")
    print(f"   Top-5 Accuracy: {acc_5:.2%}")
    
    # Save Results
    report = classification_report(y_true, y_pred, output_dict=True)
    pd.DataFrame(report).transpose().to_csv(os.path.join(config.RESULT_DIR, "classification_report.csv"))
    
    plot_results(history, config.RESULT_DIR)
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm, cmap='Blues', xticklabels=False, yticklabels=False)
    plt.savefig(os.path.join(config.RESULT_DIR, "confusion_matrix.png"))
    
    print(f"✅ All results saved to {config.RESULT_DIR}")

if __name__ == "__main__":
    import sys
    root_arg = sys.argv[1] if len(sys.argv) > 1 else "."
    main(root_arg)

In [ ]:
if DATA_ROOT:
    !python src/train.py "{DATA_ROOT}"
else:
    print("❌ Error: DATA_ROOT not found. Run Cell 1 first.")